In [25]:
import urllib2, re, os
import bs4
import pandas as pd
from tqdm import tqdm_notebook
import urllib
import numpy as np

In [3]:
SVO_BASE_URL = 'http://svo2.cab.inta-csic.es/svo/theory/fps'

In [4]:
def generate_facilities_list():
    svo_html = urllib2.urlopen(SVO_BASE_URL).read()
    svo_soup = bs4.BeautifulSoup(svo_html)
    observatory_links = svo_soup.findAll('a', attrs={'href':re.compile('gname')})
    observatory_list = pd.DataFrame(index=range(len(observatory_links)), columns=['facility', 'url'])
    for i, obs_link in enumerate(observatory_links):
        observatory_list.loc[i] = obs_link.text, obs_link.attrs['href']
    return observatory_list

def generate_instruments_list(observatory_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    return observatory_soup.findAll('a', attrs={'href':re.compile('index\.php\?mode=browse&gname=.+?&gname2=.+?')})
#    filter_soup = observatory_soup.findAll('table')[8]
#    filter_table = pd.read_html(str(filter_soup), header=0)[0]
#    return filter_table

def generate_filter_list(instrument_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    filter_soup = observatory_soup.findAll('table')[8]
    filter_table = pd.read_html(str(filter_soup), header=0)[0]
    return filter_table

In [47]:
facilities_list = generate_facilities_list()

In [52]:
facility.url

'index.php?mode=browse&gname=2MASS'

In [70]:
tqdm_notebook?

In [71]:
all_filters = None
for i, facility in tqdm_notebook(facilities_list.iterrows(), desc='Facilities', total=len(facilities_list)):
    #print "Working on {0}".format(facility.facility)
    instruments = generate_instruments_list(os.path.join(SVO_BASE_URL, facility.url))
    if len(instruments) == 0:
        #print "{0} does not have separate instruments".format(facility.facility)
        filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, facility.url))
        if all_filters is None:
            all_filters = filter_list
        else:
            all_filters = all_filters.append(filter_list)
    else:
        for instrument in tqdm_notebook(instruments, desc='Instruments', leave=False):
            #print "Working on {0}".format(instrument.text)
            filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, instrument.attrs['href']))
            if all_filters is None:
                all_filters = filter_list
            else:
                all_filters = all_filters.append(filter_list)


In [67]:
all_filters[all_filters['Filter ID'].str.startswith('AKARI')]

,Filter ID,Î»mean,Î»eff,Î»min,Î»max,Weff,ZP (Jy),Obs. Facility,Instrument,Description
0,AKARI/IRC.N2,24106.1,22684.3,16800,29497,6973.8,559.8,AKARI,IRC,IRC NIR N2
1,AKARI/IRC.N3,32761.3,31302.7,25370,39870,8605.4,325.5,AKARI,IRC,IRC NIR N3
2,AKARI/IRC.N4,44704.0,42505.2,33991,57514,15278.9,184.4,AKARI,IRC,IRC NIR N4
3,AKARI/IRC.S7,73059.1,69539.0,55236,87426,17325.8,72.5,AKARI,IRC,IRC MIR-S S7
4,AKARI/IRC.S9W,92182.0,82283.6,58461,121880,41027.1,47.5,AKARI,IRC,IRC MIR-S S9W
5,AKARI/IRC.S11,109489.1,101932.1,82698,152993,41042.6,33.7,AKARI,IRC,IRC MIR-S S11
6,AKARI/IRC.L15,161620.4,152310.5,122114,214681,59787.6,15.5,AKARI,IRC,IRC MIR-L L15
7,AKARI/IRC.L18W,198065.7,176094.9,136082,286691,99690.1,10.7,AKARI,IRC,IRC MIR-L L18W
8,AKARI/IRC.L24,233524.4,227526.4,184025,287037,53337.8,7.4,AKARI,IRC,IRC MIR-L L24


In [39]:
generate_instrument_list(os.path.join(SVO_BASE_URL, observatory_list.iloc[13, 1]))

[<a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Cape">Cape</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Cousins">Cousins</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=DDO">DDO</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Eggen">Eggen</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Geneva">Geneva</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=JHKLMN">JHKLMN</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Johnson">Johnson</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Stromgren">Stromgren</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Vilnius">Vilnius</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Walraven">Walraven</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Washington">Washington</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=WBVR">WBVR</a>]

In [43]:
x = urllib2.urlopen('http://svo2.cab.inta-csic.es/svo/theory/fps/getdata.php?format=ascii&id=2MASS/2MASS.J').read()

# Saving the data

In [13]:
tab = pd.read_hdf('svo_all_filters.h5').reset_index()

In [17]:
tab.

Index([        u'index',     u'Filter ID',        u'Î»mean',         u'Î»eff',
               u'Î»min',         u'Î»max',          u'Weff',       u'ZP (Jy)',
       u'Obs. Facility',    u'Instrument',   u'Description'],
      dtype='object')

In [26]:
for i in np.arange(len(tab)):
    ll = tab.loc[i]
    filt_id = ll['Filter ID']
    facility = ll['Obs. Facility']
    if not os.path.exists(facility):
        os.makedirs(facility)
    filename = filt_id.split('/')
    outfile = os.path.join('SVO/'+facility,filename[-1])
    url = 'http://svo2.cab.inta-csic.es/svo/theory/fps/getdata.php?format=ascii&id='+filt_id
    print(outfile)
    urllib.urlretrieve(url, outfile)


2MASS/2MASS.J
2MASS/2MASS.H
2MASS/2MASS.Ks
AAO/AAO.aao21
AAO/AAO.aao48
AAO/AAO.aao70
AAO/AAO.aao71
AAO/AAO.aao72
AAO/AAO.aao49
AAO/AAO.aao1
AAO/AAO.aao44
AAO/AAO.aao22
AAO/AAO.aao5
AAO/AAO.aao53
AAO/AAO.aao90
AAO/AAO.aao60
AAO/AAO.aao8
AAO/AAO.aao73
AAO/AAO.aao63
AAO/AAO.aao54
AAO/AAO.aao62
AAO/AAO.aao2
AAO/AAO.aao50
AAO/AAO.aao45
AAO/AAO.aao23
AAO/AAO.aao74
AAO/AAO.aao55
AAO/AAO.aao91
AAO/AAO.aao56
AAO/AAO.aao46
AAO/AAO.aao24
AAO/AAO.aao57
AAO/AAO.aao14
AAO/AAO.aao6
AAO/AAO.aao80
AAO/AAO.aao58
AAO/AAO.aao3
AAO/AAO.aao9
AAO/AAO.aao81
AAO/AAO.aao83
AAO/AAO.aao92
AAO/AAO.aao47
AAO/AAO.aao84
AAO/AAO.aao25
AAO/AAO.aao7
AAO/AAO.aao85
AAO/AAO.aao4
AAO/AAO.aao86
AAO/AAO.aao93
AKARI/FIS.N60
AKARI/FIS.WIDE-S
AKARI/FIS.WIDE-L
AKARI/FIS.N160
AKARI/IRC.N2
AKARI/IRC.N3
AKARI/IRC.N4
AKARI/IRC.S7
AKARI/IRC.S9W
AKARI/IRC.S11
AKARI/IRC.L15
AKARI/IRC.L18W
AKARI/IRC.L24
Astrosat/UVIT.F148W
Astrosat/UVIT.F148Wa
Astrosat/UVIT.F154W
Astrosat/UVIT.F169M
Astrosat/UVIT.F172M
Astrosat/UVIT.N219M
Astrosat/UVIT.N

In [ ]:
for i, data in tab.iterrows():
    break